# BioMedVis Challenge 2024

## Package Installation to Set Up the Environment:

In [1]:
pip install vitessce anywidget uvicorn starlette ome-zarr ome_types

Note: you may need to restart the kernel to use updated packages.


## Accessing the Data to create configurations

In [2]:
import dask.array as da
import zarr
import requests
import io
from ome_zarr.io import parse_url
import ome_types

1. **Zarr** is a format for the storage of chunked, compressed, N-dimensional arrays
2. **daskArray** is created using Dask, which is a parallel computing library that integrates with Python's existing libraries like NumPy. Here, da.from_zarr() loads data from the Zarr store into a Dask array. The component="3" parameter likely specifies which part or subset of the Zarr data to load, perhaps referring to a specific variable or dataset within the Zarr hierarchy. The comment # t=0, high-res suggests that this component corresponds to a specific timepoint (t=0) and is high-resolution data.

In [3]:
path = "https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0"
root = parse_url(path, mode="w")
store = root.store
daskArray = da.from_zarr(store, component="3") # t=0, high-res
daskArray

dask.array<from-zarr, shape=(1, 70, 194, 688, 1363), dtype=>u2, chunksize=(1, 1, 1, 688, 1024), chunktype=numpy.ndarray>

1. likely imaging data given the context of melanoma research
2. ```channel_names = [c.name for c in ome_xml.images[0].pixels.channels]```  
This line uses a list comprehension to iterate through the channels of the first image's pixels object in the parsed ome_xml data structure. Each channel often represents different data streams (like different dyes in microscopy), and c.name is used to get the name of each channel. The result is a list of channel names.

In [4]:
import requests
response = requests.get("https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/OME/METADATA.ome.xml")
data = response.text
ome_xml = ome_types.from_xml(response.text.replace("Â",""))
channel_names = [c.name for c in ome_xml.images[0].pixels.channels]
# Listing all the Channels in the Volume(total 70 channels)
channel_names

['Hoechst',
 "5'hmC",
 'MX1',
 'MART1',
 'Hoechst',
 'CD3E (do not use)',
 'MHC-I',
 'SOX10',
 'Hoechst',
 'S100B',
 'MITF',
 'GranzymeB (do not use)',
 'Hoechst',
 'pan-cytokeratin',
 'lamin-ABC',
 'PDL1',
 'Hoechst',
 'PD1 (do not use)',
 'S100A',
 'CD31',
 'Hoechst',
 'CD206',
 'pMLC2',
 'CD11b (do not use)',
 'Hoechst',
 'CD4',
 'LAG3',
 'CD20',
 'Hoechst',
 'PRAME',
 'CD163',
 'IRF1',
 'Hoechst',
 'B-catenin',
 'CD3E',
 'CD8a',
 'Hoechst',
 'CD11b',
 'FOXP3',
 'PD1',
 'Hoechst',
 'Ki67',
 'CD11c',
 'COX-IV',
 'Hoechst',
 'LysozymeC',
 'SOX9',
 'PMEL',
 'CD103',
 'Hoechst',
 'CyclinD1',
 'BAF1',
 'Hoechst',
 'B-actin',
 'Mast cell tryptase',
 'CD15',
 'Podoplanin',
 'Hoechst',
 'B-tubulin',
 'Catalase',
 'y-H2AX',
 'Hoechst',
 'E-cadherin',
 'Vimentin',
 'Neurofilament L (do not use)',
 'GranzymeB',
 'Hoechst',
 'MHC-II',
 'H3K27me3',
 'Collagen (SHG)']

In [5]:
# Reading the Physical Size of the Sample
ome_xml.images[0].pixels.physical_size_x, ome_xml.images[0].pixels.physical_size_y, ome_xml.images[0].pixels.physical_size_z

(0.13999999999999999, 0.14, 0.28)

In [6]:
# Getting the data from Channel CD31:
data = daskArray[0,19,:,:,:].compute()
data.shape

(194, 688, 1363)

# Visualization of volumetric imaging data


Importing the necessary libraries:

In [7]:
from vitessce import (
    VitessceConfig,
    CoordinationLevel as CL,
    get_initial_coordination_scope_prefix,
)
from os.path import join
import ipywidgets

## Getting a first view of the Data:

In [8]:
vc = VitessceConfig(schema_version="1.0.16", name="BioMedVis Challenge")
dataset = vc.add_dataset(name="Blood Vessel", uid="bv").add_file(
    url="https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0",
    file_type="image.ome-zarr"
)

spatial = vc.add_view("spatialBeta", dataset=dataset)
lc = vc.add_view("layerControllerBeta", dataset=dataset)

#Spatial Target X,Y,Z are used for translating the viewpoint to a certain point in the volume
#These coordiantes are given in the best resolution level
vc.link_views_by_dict([spatial, lc], {
    "spatialTargetZ": 0,
    "spatialTargetT": 0,
    "spatialZoom": -3.54,
    "spatialTargetX": 5230,
    "spatialTargetY": 2880,
    "spatialTargetZ": 0,
    "spatialRenderingMode": "3D",
    "imageLayer": CL([
      {
        "spatialTargetResolution": 3,
        "spatialLayerOpacity": 1.0,
        "spatialLayerVisible": True,
        "photometricInterpretation": "BlackIsZero",
        "imageChannel": CL([
          {
            "spatialTargetC": 0,
            "spatialChannelColor": [255, 255,255],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
          }])
      }
    ])
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "image"))

vc.layout(spatial | lc);
vw = vc.widget(js_package_version="3.4.5", remount_on_uid_change=False)
vw

VitessceWidget(config={'version': '1.0.16', 'name': 'BioMedVis Challenge', 'description': '', 'datasets': [{'u…

# Visualizing Regions of Interest (ROI)

Based on the prior analysis we can define a viewpoint showing an area of interest based on prior analysis in: https://doi.org/10.1101/2023.11.10.566670

```
vc.link_views_by_dict([spatial, lc], {
    # Multiple settings for targeting and visual attributes...
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "image"))
```
This section links the spatial and layer controller views and sets various parameters such as targeting coordinates (spatialTargetX, spatialTargetY, spatialTargetZ), zoom level (spatialZoom), rendering mode (spatialRenderingMode), and characteristics of different channels in the image data (like color, visibility, opacity, and window for channel adjustment).

In [9]:
vc = VitessceConfig(schema_version="1.0.16", name="BioMedVis Challenge")
dataset = vc.add_dataset(name="Blood Vessel", uid="bv").add_file(
    url="https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0",
    file_type="image.ome-zarr"
)

spatial = vc.add_view("spatialBeta", dataset=dataset)
lc = vc.add_view("layerControllerBeta", dataset=dataset)

#Spatial Target X,Y,Z are used for translating the viewpoint to a certain point in the volume
#These coordiantes are given in the best resolution level
vc.link_views_by_dict([spatial, lc], {
    "spatialTargetZ": 0,
    "spatialTargetT": 0,
    "spatialZoom": -1.1,
    "spatialTargetX": 2914,
    "spatialTargetY": 1267,
    "spatialTargetZ": 0,
    "spatialRenderingMode": "3D",
    "imageLayer": CL([
      {
        "spatialTargetResolution": 3,
        "spatialLayerOpacity": 1.0,
        "spatialLayerVisible": True,
        "photometricInterpretation": "BlackIsZero",
        "imageChannel": CL([
          {
            "spatialTargetC": 9,
            "spatialChannelColor": [255, 125, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [0,9486]
          },
          {
            "spatialTargetC": 19,
            "spatialChannelColor": [0, 255, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [666,21313]
          },
          {
            "spatialTargetC": 69,
            "spatialChannelColor": [255, 255, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [6,34]
          },
          {
            "spatialTargetC": 21,
            "spatialChannelColor": [255, 0, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [1638,36287]
          },
        ]),
      }
    ])
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "image"))

vc.layout(spatial | lc);

vw = vc.widget(js_package_version="3.4.5", remount_on_uid_change=False)
vw

VitessceWidget(config={'version': '1.0.16', 'name': 'BioMedVis Challenge', 'description': '', 'datasets': [{'u…

In [10]:
vc = VitessceConfig(schema_version="1.0.16", name="BioMedVis Challenge")
dataset = vc.add_dataset(name="Blood Vessel", uid="bv").add_file(
    url="https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0",
    file_type="image.ome-zarr"
)

spatial = vc.add_view("spatialBeta", dataset=dataset)
lc = vc.add_view("layerControllerBeta", dataset=dataset)

#Spatial Target X,Y,Z are used for translating the viewpoint to a certain point in the volume
#These coordiantes are given in the best resolution level
vc.link_views_by_dict([spatial, lc], {
    "spatialTargetZ": 0,
    "spatialTargetT": 0,
    "spatialZoom":  -1.91661,
    "spatialTargetX": 5254,
    "spatialTargetY": 2834,
    "spatialTargetZ": 0,
    "spatialRenderingMode": "3D",
    "imageLayer": CL([
      {
        "spatialTargetResolution": 3,
        "spatialLayerOpacity": 1.0,
        "spatialLayerVisible": True,
        "photometricInterpretation": "BlackIsZero",
        "imageChannel": CL([
          {
            "spatialTargetC": 9,
            "spatialChannelColor": [255, 125,0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [0,9486]
          },
          {
            "spatialTargetC": 19,
            "spatialChannelColor": [0, 0,255],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [888,6882]
          },
          {
            "spatialTargetC": 69,
            "spatialChannelColor": [255, 255, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [6,34]
          },
          {
            "spatialTargetC": 21,
            "spatialChannelColor": [255, 0,0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [0,6552]
          },
          {
            "spatialTargetC": 29,
            "spatialChannelColor": [255,0,255],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [475,2983]
          },
          {
            "spatialTargetC": 3,
            "spatialChannelColor": [0, 255,0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [1287,14825]
          },
        ]),
      }
    ])
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "image"))

vc.layout(spatial | lc);
vw = vc.widget(js_package_version="3.4.5", remount_on_uid_change=False)
vw

VitessceWidget(config={'version': '1.0.16', 'name': 'BioMedVis Challenge', 'description': '', 'datasets': [{'u…

# Reading Visualization Parameters:

In [11]:
vw.config["coordinationSpace"]["spatialTargetX"]

{'init_bv_image_0': 5254}

In [12]:
vw.config["coordinationSpace"]["spatialTargetY"]

{'init_bv_image_0': 2834}

In [13]:
vw.config["coordinationSpace"]["spatialTargetZ"]

{'init_bv_image_0': 0}

In [14]:
vw.config["coordinationSpace"]["spatialZoom"]

{'init_bv_image_0': -1.91661}

In [15]:
vw.config["coordinationSpace"]

{'dataset': {'A': 'bv'},
 'spatialTargetZ': {'init_bv_image_0': 0},
 'spatialTargetT': {'init_bv_image_0': 0},
 'spatialZoom': {'init_bv_image_0': -1.91661},
 'spatialTargetX': {'init_bv_image_0': 5254},
 'spatialTargetY': {'init_bv_image_0': 2834},
 'spatialRenderingMode': {'init_bv_image_0': '3D'},
 'imageLayer': {'init_bv_image_0': '__dummy__'},
 'spatialTargetResolution': {'init_bv_image_0': 3},
 'spatialLayerOpacity': {'init_bv_image_0': 1.0},
 'spatialLayerVisible': {'init_bv_image_0': True},
 'photometricInterpretation': {'init_bv_image_0': 'BlackIsZero'},
 'imageChannel': {'init_bv_image_0': '__dummy__',
  'init_bv_image_1': '__dummy__',
  'init_bv_image_2': '__dummy__',
  'init_bv_image_3': '__dummy__',
  'init_bv_image_4': '__dummy__',
  'init_bv_image_5': '__dummy__'},
 'spatialTargetC': {'init_bv_image_0': 9,
  'init_bv_image_1': 19,
  'init_bv_image_2': 69,
  'init_bv_image_3': 21,
  'init_bv_image_4': 29,
  'init_bv_image_5': 3},
 'spatialChannelColor': {'init_bv_image_0'